# DEL 3.7 - Oppgaver med _restcountries_ (del 2)

Før vi ser videre på api-et fra Yr, skal vi se litt på behandling, strukturering og validering av data.


In [1]:
# Importer bibliotekene vi trenger
import requests
from urllib.parse import quote, urlencode, parse_qs
import re
import pandas as pd
import matplotlib.pyplot as plt
from pydantic import BaseModel, field_validator, Field

# Dependencies for automatiske tester
import ipytest
import pytest
import warnings
from unittest.mock import patch, Mock

ipytest.autoconfig()

In [2]:
class CountryNameEntry(BaseModel):
    common: str
    official: str


class CountryName(CountryNameEntry):
    nativeName: dict[str, CountryNameEntry]


class Currency(BaseModel):
    name: str
    symbol: str

class PostalCode(BaseModel):
    format: str
    regex: str

class Country(BaseModel):
    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]
    postalCode: PostalCode

## Oppgave 3.7-1a

I denne oppgaven skal du hente ut regex-mønster for alle land i Europa. Den
har to deler - først skal du lage en funksjon som henter data fra apiet. I
del to skal du behandle responsdata fra apiet.

**DEL A**

For å hente ut data for en region må du bruke et annet endepunkt for api-et enn
det som vi har brukt for å hente ut data om ett land. Du kan se på funksjonen
`hent_landdata` definert tidligere, for hjelp til å skrive en ny hjelpefunksjon
`hent_regiondata`. 
- Denne funksjonen skal ha ett påkrevd parameter, navn, som er
navnet på regionen. 
- Den kan også ha ett valgfritt parameter, feltnavn. Dette 
parameteret skal ta inn en liste over de feltnavnene som skal hentes for hvert 
land i regionen. 
  - Dersom du ikke får dette til å fungere, kan du la være å implementere det
    valgfrie parameteret

For å løse denne oppgaven må du sjekke ut api-ets
[dokumentasjon](https://restcountries.com/). Let etter _region_ og
*filter_response*. 


In [19]:
def hent_regiondata(navn: str, feltnavn: list[str] | None = None) -> list[dict]:
    navn = quote(navn) 
    feltnavn = ",".join(feltnavn) if feltnavn else None
    url = f"https://restcountries.com/v3.1/region/{navn}?fields={feltnavn}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Feil ved henting av data fra API")

Kjør cellen under for å sjekke svaret ditt:

In [18]:
%%ipytest --tb=short 

@patch('requests.get') 
def test_hent_regiondata_url(mock_get):
    # with patch('requests.get') as mock_get:
    mock_response = Mock()
    mock_response.json.return_value = {}
    mock_response.status_code = 200
    mock_get.return_value = mock_response
    _ = hent_regiondata("unknown region", ["name", "population"])
    args, kwargs = mock_get.call_args
    url = args[0] or kwargs.get('url', '')
    base_url = "https://restcountries.com/v3.1/region/"
    url_rest = url[len(base_url):]
    url_region, url_params = (url_rest.split('?', 1) + [''])[:2]
    assert url.startswith(base_url), (
        "Funksjonen bygger ikke riktig URL: feil base URL"
    )
    assert url_region == quote("unknown region"), (
        "Funksjonen bygger ikke riktig URL: regionnavn ikke korrekt i URL"
    )
    if url_params:
        params_parsed = parse_qs(url_params)
        assert len(params_parsed) > 0, (
            "Funksjonen bygger ikke riktig URL: mangler parametre"
        )
        fields_params = params_parsed.get('fields', None)
        assert fields_params is not None, (
            "Funksjonen bygger ikke riktig URL: mangler 'fields' parameter"
        )
        if len(fields_params) == 1:
            fields_params = fields_params[0].split(",")
        assert 'name' in fields_params and 'population' in fields_params, (
            "Funksjonen bygger ikke riktig URL: mangler forventede feltnavn i 'fields' parameter"
        )
    else:
        warnings.warn("parameteret 'feltnavn' er ikke definert i funksjonen hent_regiondata", UserWarning)

def test_hent_regiondata_request():    
    data = hent_regiondata("europe")
    assert isinstance(data, list), (
        "Forventet liste som returverdi"
    )
    assert len(data) > 0, (
        "Funksjonen returnerer tom liste"
    )
    assert all(isinstance(item, dict) for item in data), (
        "Forventet liste av dicts som returverdi"
    )


..                                                                                           [100%]
2 passed in 0.58s


**DEL B**

I cellen under er en modell-definisjon `CountryPartial`, som er en variant av
`Country` vi har jobbet med tidligere. Her har vi kun definert egenskapene
som trengs for å løse oppgaven, `name` og `postal_code`.

Siden data ikke er like utfyllende for alle land, kan noen felter mangle når vi
etterspør data fra apiet. Derfor er det viktig å gjøre felt som ikke alltid
eksisterer eller inneholder data valgfrie (dvs: tildele egenskapene en
standardverdi). 

Under testing ser det ut til at noen felt uten data utelates helt. Dette går
fint for valideringen når vi har standardverdi `None` - da vil egenskapen få
verdien `None` som indikerer at datafeltet er tomt. Andre ganger returnerer
api-et en tom `dict` når data mangler. Dette vil lede til valideringsfeil! 
en tom `dict` er en verdi, så standardverdien vil ikke tildeles. Men siden
vi har satt typen til `PostalCode` vil vi få feil, da `PostalCode` krever
feltene `regex` og `format`. 

Dette kunne vært løst på flere måter, som å gjøre felt i `PostalCode`
valgfrie, eller type `postal_code`-egenskapen til `ContryPartial` som 
`PostalCode | dict | None`. Under brukes en _decorator_ fra `pydantic`
som kjører en egendefinert funksjon som validerer/konverterer data før
verdien til egenskapen settes. `field_validator`-dekoratørens første 
parameter er navnet på egenskapen den skal knyttes til. `mode='before'`
gjør at valideringen kjøres før verdien til egenskapen settes, som er 
nødvendig siden vi skal gjøre en konvertering av inndata. Dekoratøren
`classmethod` er en del av standardbiblioteket til `python` og betyr
at funksjonen som dekoreres blir en statisk funksjon som knyttes til
klassen, og ikke til et enkelt objekt av klassen. Du har kanskje ikke
lært om statiske metoder enda, så det kan vi ta mer om senere. 

Selve funksjonen er ganske enkel, den returnerer `None` som verdi dersom
verdien fra inndata er `None` eller en tom `dict`(`{}`). Ellers sendes
verdien til å bli behandlet av `PostalCode` modellen. Hvis vi hadde fjernet
`or value == {}` hadde funksjonen gjort akkurat det samme som den innebygde
valideringen for egenskapen! Det som gjør at vi oppnår hensikten er altså at 
vi setter egenskapens verdi til `None` også dersom inndata er `{}`

Kjør cellen under for å definere `ContryPartial` før du går videre til oppgaven.

In [20]:


class CountryPartial(BaseModel):
    name: CountryName
    postal_code: PostalCode | None = Field(default=None, alias='postalCode')

    @field_validator('postal_code', mode='before')
    @classmethod
    def validate_postal_code(cls, value):
        if value is None or value == {}:
            return None
        return PostalCode(**value)

## Oppgave 3.7-1b

Lag en funksjon, `hent_postnummer_format_europa`, som henter regex-mønster for
alle land i Europa, og returnerer data som en `dict` med landets navn som nøkkel
og regex-mønster som verdi. Bruk `CountryPartial` for å konvertere / validere data
fra `hent_regiondata`.

Bruk startkoden under og fyll ut din implementering

In [ ]:
def hent_postnummer_regex_europa() -> dict[str, str | None]:
    """Hent postnummer-format for alle europeiske land.
    
    Returns
    ---------
    dict[str, str | None]
        Ordbok med landnavn som nøkler og postnummer-format i regex som verdier.
        Dersom et land ikke har postnummer-format, skal verdien være None.
    """
    regiondata = hent_regiondata(_, feltnavn=_) # Bruk feltnavn-parameteret 
                                                # dersom du fikk dette til å virke.
                                                # Ellers kan du bare bruke regionnavn.
    regiondata_formatert: list[CountryPartial] = []
    for country_data in regiondata:
        pass # Lag CountryPartial objekter fra dataen og legg dem i regiondata_formatert listen

    postnr_data: dict[str, str | None] = {}
    for country in regiondata_formatert:
        pass # Hent navn og postnummer-format fra land i regiondata_formatert
             # og legg dem i postnr_data ordboken {landnavn: postnummer_format}
             # Husk å håndtere land der postal_code er None. Disse skal
             # lagres som {landnavn: None} i postnr_data

    return postnr_data



# Disse linjene kjører funksjonen og skriver ut resultatet,
# slik at du kan kjøre cellen for å se om det fungerer som forventet.
postnummer_regex_europa = hent_postnummer_regex_europa()
print("Postnummer format for europeiske land:")
for land, format in postnummer_regex_europa.items():
    print(f"{land}: {format}")

_Kjør cellen under for å sjekke om du har løst oppgaven riktig!_

In [ ]:
%%ipytest --tb=short
def test_hent_postnummer_regex_europa():
    result = hent_postnummer_regex_europa()
    assert isinstance(result, dict), (
        "Forventet ordbok som returverdi"
    )
    for land, format in result.items():
        assert isinstance(land, str), "Forventet str som landnavn"
        assert format is None or isinstance(format, str), (
            f"Forventet str eller None som postnummer-format for {land}"
        )
    valid_codes = {
        "United Kingdom": "EH11AA",
        "Norway": "5000",
        "Germany": "50667",
    }
    invalid_codes = {
        "United Kingdom": "12345",
        "Norway": "ABCDE",
        "Germany": "XYZ987",
    }
    for land, kode in valid_codes.items():
        pattern = result.get(land, None)
        assert pattern is not None, (
            f"Mangler postnummer-format for {land}"
        )
        assert re.match(pattern, kode), (
            f"Postnummer {kode} i {land} skal være gyldig, men "
            + "stemte ikke med format hentet fra hent_postnummer_format_europa."
        )
    for land, kode in invalid_codes.items():
        pattern = result.get(land, None)
        assert pattern is not None, (
            f"Mangler postnummer-format for {land}"
        )
        assert not re.match(pattern, kode), (
            f"Postnummer {kode} i {land} skal være ugyldig, men "
            + "stemte med format hentet fra hent_postnummer_format_europa."
        )



## Oppgave 3.7-2

I denne oppgaven skal vi bruke `pandas` for å samle informasjon om regionale data. 

For å sikre at du har en passende datamodell tilgjengelig, kan du bruke
`CountryPandas` under. Kjør cellen for at denne skal bli definert.

In [ ]:
class CountryPandas(BaseModel):
    name: CountryName
    population: int | None = None
    area: float | None = None
    region: str | None = None
    cca3: str | None = Field(default=None, min_length=3, max_length=3) # ISO 3166-1 alpha-3 kode
    currencies: dict[str, Currency] | None = None
    languages: dict[str, str] | None = None
    borders: list[str] | None = None
    start_of_week: str | None = Field(default=None, alias='startOfWeek')

**Uke-start i Nord-Amerika**

Først skal du hente data for Nord-Amerika, og laste den inn i en `DataFrame`.
For å vise hvordan man kan gjøre dette med bruk av `pydantic` modellen, 
vises dette i cellen under. Kjør cellen for å laste inn data om Nord-Amerika
i `DataFrame` med variabelnavn `north_america`

In [ ]:
# Hent data fra API
data_na_raw = hent_regiondata("north america")
# Lag CountryPandas objekter fra rådata
data_na = [CountryPandas(**land) for land in data_na_raw]

# Bruk model_dump for å konvertere til dicts, og lag DataFrame
north_america = pd.DataFrame([land.model_dump() for land in data_na])



Videre kan det være greit å velge ut kolonnene du skal jobbe med, og sørge for
at kolonnene inneholder data formatert på ønsket måte. La oss velge kolonnene 
vi trenger og ta en titt på de første verdiene:

In [ ]:
# Senere i oppgaven trenger vi kolonnene name, population og start_of_week
# så vi velger dem ut her:
north_america = north_america[["name", "population", "start_of_week"]]

# Velg ut de øverste radene med head()
# Den siste variabelverdien i en celle skrives ut som resultat i jupyter
# notebook, selv om vi ikke bruker print()
north_america.head()

Kolonnen 'name' har litt pussige verdier... Dette er `CountryName`
konvertert til `dict`. Vi ønsker bare å beholde `common` verdien.
Det kan vi ordne på følgende måte:

In [ ]:
# Apply kjører en funksjon på hver verdi i en kolonne.
# Her lager vi en ny kolonne fra "name" kolonnen,
# der vi henter ut bare det vanlige navnet (common)
# Så overskrives den opprinnelige "name" kolonnen med denne nye kolonnen.
north_america["name"] = north_america["name"].apply(lambda x: x["common"])

# Skriv ut de første radene for å se resultatet
north_america.head()

Dette ser bedre ut! Da er data klar til å jobbes videre med.

### 3.7.2a - Grupper og aggreger data

Nå skal du gruppere data etter `start_of_week` og legge til tre kolonner
(aggresjoner) som viser antall, befolkningstall og en liste med alle landene som
hører til under verdien for `start_of_week`.

Resultatet skal se ca slik ut:

![](./img/ukestart_tabell.png)


> Du har kanskje vært borti `agg` og `groupby` i pandas tidligere.
> Det denne oppgaven ber om er kanskje litt mer avansert.
> 
> Sjekk ut disse artiklene om `agg` og `groupby`:
> - [g4g - pandas groupby](https://www.geeksforgeeks.org/pandas/python-pandas-dataframe-groupby/)
> - [g4g - pandas aggregate](https://www.geeksforgeeks.org/python/python-pandas-dataframe-aggregate/)
> - [g4g - rows into list](https://www.geeksforgeeks.org/python/how-to-group-dataframe-rows-into-list-in-pandas-groupby/)
> 
> Jeg anbefaler at du bruker _Named Aggregation_ når du bruker `agg`.
> Du kan finne god dokumentasjon på hvordan dette gjøres her: 
> [pandas.pydata.org](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation)

In [ ]:
# Bruk groupby og agg for å gruppere data fra north_america
data_grouped = north_america.groupby(_).agg( # Fyll inn
    _ # Fyll inn
)

# Skriver ut kolonnenavnene og tabellen
print("Kolonnenavn etter aggregering:")
print(data_grouped.columns)
print("\nTabell etter aggregering:")
data_grouped

Sjekk utskrift fra cellen over etter du har lagt inn parametre
for `agg` og `groupby`

Hvis du har brukt _Named Aggregation_ bør tabellen se ganske riktig ut allerede!
Output fra cellen over bør gi kolonnenavn som ligner:

```python
Index(['count', 'total_population', 'countries'], dtype='object')
```

Hvis du har brukt _Dictionary Aggregation_ vil kolonenavnene ligne:

```python
MultiIndex([(      'name', 'count'),
            (      'name',  'list'),
            ('population',   'sum')],
           )
```

Og hvis du ikke har angitt spesifikke kolonner for aggregeringsfunksjonene kan
du ha fått ut noe som ligner:

```python
MultiIndex([(      'name', 'count'),
            (      'name',   'sum'),
            (      'name',  'list'),
            ('population', 'count'),
            ('population',   'sum'),
            ('population',  'list')],
           )
```

Det enkleste er om du bruker _Named Aggregation_, så du kan se om du får det til
over.

Dersom du har endt opp med `MultiIndex` i stedet for `Index`, er det også fullt
mulig å behandle tabellen videre for å nå målet. Sjekk eksempelet under:

#### Eksempel: Flate ut multi-index kolonner

In [ ]:
# Disse linjene oppretter bare et eksempel DataFrame med multi-index kolonner,
# som vi kan bruke til å illustrere hvordan vi kan flate ut kolonnenavnene.

df = pd.DataFrame({
    ('sales', 'sum'): [100, 200, 300],
    ('sales', 'mean'): [50.0, 100.0, 150.0],
    ('quantity', 'sum'): [20, 40, 60],
    ('quantity', 'mean'): [10.0, 20.0, 30.0],
}, index=['a', 'b', 'c'])

print("Opprinnelig DataFrame med multi-index kolonner:")
print(df)
print("\nKolonnenavn før flattening:")
print(df.columns)

# Flate ut kolonnenavnene ved å slå sammen nivåene med map og join
# med '_' som separator
df_flat = df.copy()
df_flat.columns = df_flat.columns.map('_'.join)

print("\nDataFrame etter flattening:")
print(df_flat)
print("\nKolonnenavn etter flattening:")
print(df_flat.columns)

# Videre kan vi velge ut kolonner vi er interessert i,
# og gi dem mer beskrivende navn.
df_flat_renamed = df_flat[['sales_sum', 'sales_mean', 'quantity_sum']].rename(columns={
    'sales_sum': 'total sales',
    'sales_mean': 'average sale amount',
    'quantity_sum': 'quantity sold',
})

print("\nNy dataframe med utvalgte kolonne med nye navn:")
print(df_flat_renamed)


In [ ]:

# Ved behov, kan du flate ut / gi nye navn til kolonnene i data_grouped her:
# (se eksempelet over)
# Hvis du ikke trenger å fikse kolonnene i data_grouped, trenger du ikke
# å gjøre noe i denne cellen.

# Denne linjen skriver ut resultatet
data_grouped

### Oppgave 3.7-2b

Neste oppgave er å plotte disse data i to kakediagram. Bruk `plot`-funksjonen
til `DataFrame` for å gjøre dette.

For å få en finere figur, der begge diagrammene er i samme figur, kan vi også 
bruke `subplots` fra `matplotlib.pyplot`. Dette inkluderes i startkoden under.
Du skal fylle inn riktige parametre i de to `plot`-funksjonene for å lage to
plots:
- Kakediagram som viser uke-start verdier fordelt på antall land
- Kakediagram som viser uke-start verdier fordelt på befolkning

Det skal se ca slik ut:

![fasit - 3.7-9b (diagram)](./img/ukestart_diagram.png)

> Tips - hjelp til plotting
> 
> - [g4g - Pandas Plotting](https://www.geeksforgeeks.org/pandas/how-to-plot-a-dataframe-using-pandas/)
> - [g4g - Matplotlib Pie Chart](https://www.geeksforgeeks.org/data-science/plot-a-pie-chart-in-python-using-matplotlib/)
> - [g4g - Matplotlib Subplots](https://www.geeksforgeeks.org/python/matplotlib-pyplot-subplots-in-python/)

In [ ]:
# Denne linjen lager et figur-objekt med en rad og to kolonner
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

data_grouped.plot(
    # Fyll inn nødvendige parametre for diagram 1 her

    ax=axes[0], # Sørger for at diagrammet tegnes i den første aksen i figuren
)
data_grouped.plot(
    # Fyll inn nødvendige parametre for diagram 2 her

    ax=axes[1], # Sørger for at diagrammet tegnes i den andre aksen i figuren
)

plt.tight_layout() # Justerer layouten slik at ting ikke overlapper
plt.show() # Viser figuren med diagrammene